# Tmap 검색건수 : 정답 레이블 전처리 

관광수요를 예측하는 LSTM 모델에 지도 학습에 쓰일 정답 레이블 Tmap 검색 건수 전처리를 진행함.

[데이터셋 변수 설명] 
- 순위 = 관광지 인기 순위
- 관광지명 = 관광객의 목적지
- 도로명주소 = 관광지명 주소
- 중분류 카테고리 = 중분류로 관광지명 유형분류
- 소분류 카테고리 = 소분류로 관광지명 유형분류
- 검색건수 = 관광지명 Tmap에서 검색된 건수

[전처리 과정 설명] 
- 일간으로 다운받은 365개의 티맵건수 데이터 불러옴. 
- 필요한 열 '관광지명', '검색건수' 열만 남기고 불필요한 열 drop
- 한국관광데이터랩에서는 검색 건수 데이터를 기간별로 상위 500개만을 제공하는데, 365일 모든 일자에 대해 순위 안에 드는 데이터를 혼잡한 관광지로 판단하여 하나의 리스트로 저장함
- 혼잡한 관광지에 대해 2021년의 데이터를 일자별로 추출하여 타겟 데이터를 제외하여 가공한 데이터셋에 조인함


[기타 데이터 관련 설명] 
- 자동차 이용 방문자의 이동행태이며 대중교통 및 도보 이용 방문자의 특성과 다를 수 있음.

source : 한국관광 데이터랩 - 빅데이터 - 내비게이션 - 지역별 관광지 검색순위
https://datalab.visitkorea.or.kr/datalab/portal/bda/getTourVisitCnt.do#

In [1]:
import pandas as pd
from glob import glob

In [2]:
#glob()로 제주시 티맵검색 데이터 불러오기
file_names = glob("../Dataset/raw_data/target/target_j/*.xlsx")
tmap_jeju = pd.DataFrame() 


for file_name in file_names:
    tmap = pd.read_excel(file_name)

    tmap_jeju = pd.concat([tmap_jeju, tmap])


tmap_jeju

,순위,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수
0,1,제주국제공항,제주 제주시 공항로 2-0,기타관광,교통시설,883
1,2,동문재래시장,제주 제주시 관덕로14길 20-0,쇼핑,시장,451
2,3,함덕해수욕장,제주 제주시 조함해안로 525-0,자연관광,자연경관(하천/해양),360
3,4,협재해수욕장,제주 제주시,자연관광,자연경관(하천/해양),255
4,5,아르떼뮤지엄제주,제주 제주시 어림비로 478-0,문화관광,전시시설,233
...,...,...,...,...,...,...
500,464,짬뽕에취한날[중식],제주 제주시 귀아랑2길 4-0,음식,외국식,12
501,464,인그리드,제주 제주시 서해안로 64-0,음식,카페/찻집,12
502,464,다이소제주용담해안로점,제주 제주시 서해안로 510-0,쇼핑,기타쇼핑시설,12
503,464,빌레리조트,제주 제주시 고내로13길 79-0,숙박,콘도미니엄,12


In [3]:
#필요한 열만 추출
tmap_jeju = tmap_jeju[['관광지명', '검색건수']]
tmap_jeju

,관광지명,검색건수
0,제주국제공항,883
1,동문재래시장,451
2,함덕해수욕장,360
3,협재해수욕장,255
4,아르떼뮤지엄제주,233
...,...,...
500,짬뽕에취한날[중식],12
501,인그리드,12
502,다이소제주용담해안로점,12
503,빌레리조트,12


In [4]:
#제주도 poi 데이터 불러오기
poi_list = pd.read_excel("../Dataset/final_data/Jeju_POI_withexplanation_final.xlsx", header=0)
poi_list = poi_list[['Unnamed: 1','Unnamed: 2']]
# poi_list=poi_list[['poi','location']]
poi_list.columns = ['poi','location']
poi_list

,poi,location
0,헬로키티아일랜드,서귀포시
1,엉알해안산책로,제주시
2,1100고지습지,서귀포시
3,우도도항선선착장,제주시
4,혼인지,서귀포시
...,...,...
116,제주불빛정원,제주시
117,더마파크,제주시
118,거문오름,제주시
119,다랑쉬오름,제주시


In [5]:
#제주시 poi만 추출 
poi_jeju= poi_list[poi_list['location'] == '제주시']
poi_jeju=poi_jeju['poi']
poi_jeju

1         엉알해안산책로
3        우도도항선선착장
7           메이즈랜드
8             한라산
9             비자림
13           어승생악
16           정물오름
17          월정리해변
20            만장굴
25           산굼부리
28            비양도
32      제주4∙3평화공원
33        제주돌문화공원
35         김녕미로공원
41         이호테우해변
44     제주민속자연사박물관
48       에코랜드테마파크
50        제주해녀박물관
53          한라수목원
56        노루생태관찰원
57         세화해수욕장
59        국립제주박물관
60         협재해수욕장
63         동문재래시장
64           동백동산
65         삼양해수욕장
66      제주절물자연휴양림
68            수월봉
69            지미봉
71          금능석물원
73            관덕정
74        생각하는 정원
75       그리스신화박물관
79           한림공원
80           아부오름
81            금오름
82         김녕해수욕장
84         곽지해수욕장
85            삼성혈
90            용두암
93        종달리해안도로
95         제주공룡랜드
97          사진놀이터
100         한라생태숲
101       제주유리의 성
102        금능해수욕장
103          새별오름
104         용눈이오름
107       브릭캠퍼스제주
108        제주신산공원
109      제주별빛누리공원
111       검멀레해수욕장
112       제주현대미술관
114      신창풍차해안도로
116        제주불빛정원
117       

In [6]:
#제주시의 poi list 형태로 반환 
poi_jeju_val = poi_jeju.values
poi_jeju_list = poi_jeju_val.tolist()
poi_jeju_list

['엉알해안산책로',
 '우도도항선선착장',
 '메이즈랜드',
 '한라산',
 '비자림',
 '어승생악',
 '정물오름',
 '월정리해변',
 '만장굴',
 '산굼부리',
 '비양도',
 '제주4∙3평화공원',
 '제주돌문화공원',
 '김녕미로공원',
 '이호테우해변',
 '제주민속자연사박물관',
 '에코랜드테마파크',
 '제주해녀박물관',
 '한라수목원',
 '노루생태관찰원',
 '세화해수욕장',
 '국립제주박물관',
 '협재해수욕장',
 '동문재래시장',
 '동백동산',
 '삼양해수욕장',
 '제주절물자연휴양림',
 '수월봉',
 '지미봉',
 '금능석물원',
 '관덕정',
 '생각하는 정원',
 '그리스신화박물관',
 '한림공원',
 '아부오름',
 '금오름',
 '김녕해수욕장',
 '곽지해수욕장',
 '삼성혈',
 '용두암',
 '종달리해안도로',
 '제주공룡랜드',
 '사진놀이터',
 '한라생태숲',
 '제주유리의 성',
 '금능해수욕장',
 '새별오름',
 '용눈이오름',
 '브릭캠퍼스제주',
 '제주신산공원',
 '제주별빛누리공원',
 '검멀레해수욕장',
 '제주현대미술관',
 '신창풍차해안도로',
 '제주불빛정원',
 '더마파크',
 '거문오름',
 '다랑쉬오름',
 '화조원']

In [7]:
#제주시 poi리스트만을 대상으로 하여 티맵 검색 건수 데이터 색인
tmap_jeju=tmap_jeju.loc[tmap_jeju['관광지명'].isin(poi_jeju_list)]
tmap_jeju

,관광지명,검색건수
1,동문재래시장,451
3,협재해수욕장,255
5,이호테우해변,169
8,신창풍차해안도로,151
10,새별오름,131
...,...,...
242,제주불빛정원,23
245,아부오름,22
247,동백동산,22
250,검멀레해수욕장,22


In [8]:
#혼잡한 제주 poi확인

tmap_jeju['관광지명'].value_counts() == 365

동문재래시장         True
한림공원           True
이호테우해변         True
신창풍차해안도로       True
새별오름           True
용두암            True
곽지해수욕장         True
월정리해변          True
김녕해수욕장         True
비자림            True
협재해수욕장         True
금오름            True
한라수목원          True
세화해수욕장         True
삼양해수욕장         True
금능해수욕장         True
에코랜드테마파크      False
제주절물자연휴양림     False
산굼부리          False
만장굴           False
수월봉           False
김녕미로공원        False
제주공룡랜드        False
화조원           False
메이즈랜드         False
아부오름          False
다랑쉬오름         False
브릭캠퍼스제주       False
제주해녀박물관       False
제주돌문화공원       False
제주민속자연사박물관    False
용눈이오름         False
더마파크          False
제주불빛정원        False
검멀레해수욕장       False
제주현대미술관       False
국립제주박물관       False
한라산           False
삼성혈           False
한라생태숲         False
종달리해안도로       False
비양도           False
사진놀이터         False
그리스신화박물관      False
동백동산          False
지미봉           False
어승생악          False
노루생태관찰원       False
엉알해안산책로       False
관덕정           False


In [9]:
#혼잡한 제주 poi만 출력

confusion_jeju = ['한림공원','곽지해수욕장','세화해수욕장','금능해수욕장','협재해수욕장','비자림','월정리해변',
'신창풍차해안도로','김녕해수욕장','삼양해수욕장','이호테우해변','금오름','동문재래시장','한라수목원','용두암','새별오름']

#for confusion_poi in confusion_jeju:
 #    df = tmap_jeju[tmap_jeju['관광지명'] == '한림공원']
  #   df = df['검색건수']
   #  df = df.reset_index()
    #temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'minimum temperature'}, inplace=True)

    #tmap_jeju = pd.concat([tmap_jeju, tmap])

In [10]:
#기존 데이터셋 불러오기
#파일은 poi 별로 불러와야 함.

df = tmap_jeju[tmap_jeju['관광지명'] == '새별오름']
df = df['검색건수']
df = df.reset_index()
df
#df = df.drop(['index'], axis=1, inplace=True)
#df
#df.to_csv('./Hallim_Park.csv',index=False)

,index,검색건수
0,10,131
1,11,129
2,14,97
3,6,87
4,4,90
...,...,...
360,10,210
361,9,246
362,6,287
363,9,249


In [153]:
#기존 데이터셋 불러오기
df2 = pd.read_csv('./weather_final_with_buzz/제주시-새별오름.csv')
df2

,visit,date,buzz,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type
0,새별오름,2021-01-01,44,1,1,-1,5,75,30,0
1,새별오름,2021-01-02,37,2,0,0,4,75,30,0
2,새별오름,2021-01-03,38,2,0,3,7,75,30,0
3,새별오름,2021-01-04,52,0,0,-1,5,75,0,0
4,새별오름,2021-01-05,36,0,0,-4,2,75,30,0
...,...,...,...,...,...,...,...,...,...,...
360,새별오름,2021-12-27,69,0,0,3,6,80,30,0
361,새별오름,2021-12-28,51,0,0,2,8,85,30,0
362,새별오름,2021-12-29,50,0,0,0,4,75,30,0
363,새별오름,2021-12-30,54,0,0,-1,2,65,60,2


In [154]:
#기존 데이터셋과 정답레이블 병합
df = pd.concat([df2, df],axis=1)
df

,visit,date,buzz,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type,index,검색건수
0,새별오름,2021-01-01,44,1,1,-1,5,75,30,0,22,180
1,새별오름,2021-01-02,37,2,0,0,4,75,30,0,37,95
2,새별오름,2021-01-03,38,2,0,3,7,75,30,0,23,190
3,새별오름,2021-01-04,52,0,0,-1,5,75,0,0,45,102
4,새별오름,2021-01-05,36,0,0,-4,2,75,30,0,5,160
...,...,...,...,...,...,...,...,...,...,...,...,...
360,새별오름,2021-12-27,69,0,0,3,6,80,30,0,27,180
361,새별오름,2021-12-28,51,0,0,2,8,85,30,0,4,571
362,새별오름,2021-12-29,50,0,0,0,4,75,30,0,25,154
363,새별오름,2021-12-30,54,0,0,-1,2,65,60,2,2,693


In [155]:
#칼러명 재정의
df.rename(columns={'visit' : 'poi',
                   '검색건수' : 'num_of_search'}, inplace=True)
df.drop(['index'], axis=1, inplace=True)
df

,poi,date,buzz,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type,num_of_search
0,새별오름,2021-01-01,44,1,1,-1,5,75,30,0,180
1,새별오름,2021-01-02,37,2,0,0,4,75,30,0,95
2,새별오름,2021-01-03,38,2,0,3,7,75,30,0,190
3,새별오름,2021-01-04,52,0,0,-1,5,75,0,0,102
4,새별오름,2021-01-05,36,0,0,-4,2,75,30,0,160
...,...,...,...,...,...,...,...,...,...,...,...
360,새별오름,2021-12-27,69,0,0,3,6,80,30,0,180
361,새별오름,2021-12-28,51,0,0,2,8,85,30,0,571
362,새별오름,2021-12-29,50,0,0,0,4,75,30,0,154
363,새별오름,2021-12-30,54,0,0,-1,2,65,60,2,693


In [156]:
#csv로 저장
df.to_csv('./jeju_search/saebyeol_oreum.csv',index=False)